# How to Deal with Missing Data in Python

## The Employee Dataset

We will be working with a very small Employee Dataset for this tutorial. Download the dataset in csv format from my Github repo and store it in your current working directory: [employees.csv](https://github.com/ChaitanyaBaweja/Programming-Tutorials/tree/master/Missing-Data-Pandas)

Let’s import this dataset into Python and take a look at it. 


In [2]:
# Importing libraries
import pandas as pd
import numpy as np

# Read csv file into a pandas dataframe
df = pd.read_csv("employees.csv")

# Prints out the first few rows
print(df.head())


  First Name  Gender  Salary Bonus % Senior Management             Team
0    Douglas    Male   97308   6.945              TRUE        Marketing
1     Thomas    Male   61933     NaN              TRUE              NaN
2      Maria  Female  130590  11.858             FALSE          Finance
3      Jerry    Male     NaN    9.34              TRUE          Finance
4      Larry    Male  101004   1.389              TRUE  Client Services


There are 1000 columns with 8 variables. You can get some basic statistics out using the `.dtypes` and `.describe()` method.

In [3]:
print(df.dtypes)
print(df.describe())

First Name           object
Gender               object
Salary               object
Bonus %              object
Senior Management    object
Team                 object
dtype: object
       First Name  Gender  Salary Bonus % Senior Management             Team
count         931     852     998     997               932              957
unique        201       3     993     968                 4               13
top       Marilyn  Female  121160  12.182              TRUE  Client Services
freq           11     428       2       3               467              105


You would notice that the dtypes of all the columns is object. This shouldn't be the case for Salary, Senior Management and Bonus. This happens because we have **corrupt values in these columns**. Once we handle these missing values, we will convert these to the required type using `.astype()` method. 

## How to mark invalid/ corrupt values as missing

Pandas treat None and NaN as essentially interchangeable for indicating missing or null values. Other values like na and ? are not recognized by Pandas by default. Let’s focus on the Salary Column. 

In [4]:
print('Salary')
print(df['Salary'].head(10))

Salary
0     97308
1     61933
2    130590
3       NaN
4    101004
5    115163
6     65476
7     45906
8       NaN
9    139852
Name: Salary, dtype: object


In the 8th row there’s a missing value and in the 3rd row there is a NA, which Pandas automatically fills with NaN. But what happens with other symbols like ?, n.a., etc. Let's look at the Gender column.

In [5]:
print(df['Gender'].head(10))

0      Male
1      Male
2    Female
3      Male
4      Male
5      n.a.
6    Female
7    Female
8       NaN
9    Female
Name: Gender, dtype: object


We notice that n.a. isn't converted to NaN and remains in its original form. 
We can pass these formats in the `.read_csv()` method to allow Pandas to recognize them as corrupt values. Take a look:

In [6]:
# a list with all missing value formats
missing_value_formats = ["n.a.","?","NA","n/a", "na", "--"]
df = pd.read_csv("employees.csv", na_values = missing_value_formats)

#print gender again
print(df['Gender'].head(10))

0      Male
1      Male
2    Female
3      Male
4      Male
5       NaN
6    Female
7    Female
8       NaN
9    Female
Name: Gender, dtype: object


Till now, our missing values had unique identifiers which, made them pretty easy to catch. But what happens when we get an invalid data type. I have designed a function that allows me to check for invalid data types in a column.

In [7]:
import pandas as pd

missing_value_formats = ["n.a.","?","NA","n/a", "na", "--"]
df = pd.read_csv("employees.csv", na_values = missing_value_formats)

def make_int(i):
    try:
        return int(i)
    except:
        return pd.np.nan

# apply make_int function to the entire series using map
df['Salary'] = df['Salary'].map(make_int)
print(df['Salary'].head())


0     97308.0
1     61933.0
2    130590.0
3         NaN
4    101004.0
Name: Salary, dtype: float64


### Marking missing values using isnull and notnull

In Pandas, we have two functions for marking missing values:   
- `isnull()` function to mark all of the NaN values in the dataset as True  
- `notnull()` to mark all of the NaN values in the dataset as False.


In [8]:
print(df['Gender'].isnull().head(10)) # NaN values are marked True
print(df['Gender'].notnull().head(10)) # non-NaN values are marked True


0    False
1    False
2    False
3    False
4    False
5     True
6    False
7    False
8     True
9    False
Name: Gender, dtype: bool
0     True
1     True
2     True
3     True
4     True
5    False
6     True
7     True
8    False
9     True
Name: Gender, dtype: bool


We can use the outputs of the `isnull` and `notnull` function for filtering. Let’s print all those rows of the database for which Gender is not missing. 

In [9]:
# returns True on indices for which Gender is not NaN
null_filter = df['Gender'].notnull()
print(df[null_filter].head())

  First Name  Gender    Salary  Bonus % Senior Management             Team
0    Douglas    Male   97308.0    6.945              True        Marketing
1     Thomas    Male   61933.0      NaN              True              NaN
2      Maria  Female  130590.0   11.858             False          Finance
3      Jerry    Male       NaN    9.340              True          Finance
4      Larry    Male  101004.0    1.389              True  Client Services


### Missing Value Statistics
`isnull` and `notnull` can also be used to summarize missing values. 

To check if there are any missing values in our data frame:

In [22]:
print(df.isnull().values.any())

True


Total number of missing values per column:

In [23]:
print(df.isnull().sum())

First Name            70
Gender               149
Salary                 5
Bonus %                4
Senior Management     71
Team                  48
dtype: int64


### How to remove rows with missing values

Pandas library provides the `dropna()` function that can be used to drop either columns or rows with missing data. 

In the example below, we use dropna() to remove all rows with missing data:

In [10]:
# drop all rows with NaN values
new_df = df.dropna(axis=0)

# check if we have any NaN values in our dataset
print(new_df.isnull().values.any())


False


We can also use the `how` parameter.
- `how = 'any'`: at least one value must be null.
- `how = 'all'`: all values must be null.

In [11]:
# drop all rows with atleast one NaN
new_df = df.dropna(axis = 0, how ='any')  

# drop all rows with all NaN
new_df = df.dropna(axis = 0, how ='all')

# drop all columns with atleast one NaN
new_df = df.dropna(axis = 1, how ='any')

# drop all columns with all NaN
new_df = df.dropna(axis = 1, how ='all')


### Replacing NaNs with a single constant value

We will use `fillna()` to replace missing values in the **Salary** column with 0.


In [13]:
df['Salary'].fillna(0)

0       97308.0
1       61933.0
2      130590.0
3           0.0
4      101004.0
         ...   
995    132483.0
996     42392.0
997     96914.0
998     60500.0
999    129949.0
Name: Salary, Length: 1000, dtype: float64

We can also do the same for categorical variables like **Gender**.

In [14]:
df['Gender'].fillna('No Gender')

0           Male
1           Male
2         Female
3           Male
4           Male
         ...    
995    No Gender
996         Male
997         Male
998         Male
999         Male
Name: Gender, Length: 1000, dtype: object

### Replacing NaNs with the value from the previous row

This is a common approach when filling missing values in image data. We use `method = 'pad'`. Let us try the same for the Salary Column:

In [15]:
df['Salary'].fillna(method='pad')

0       97308.0
1       61933.0
2      130590.0
3      130590.0
4      101004.0
         ...   
995    132483.0
996     42392.0
997     96914.0
998     60500.0
999    129949.0
Name: Salary, Length: 1000, dtype: float64

### Replacing NaNs with the value from the next row
We use `method = 'bfill'`.

In [16]:
df['Salary'].fillna(method='bfill')

0       97308.0
1       61933.0
2      130590.0
3      101004.0
4      101004.0
         ...   
995    132483.0
996     42392.0
997     96914.0
998     60500.0
999    129949.0
Name: Salary, Length: 1000, dtype: float64

### Replacing NaNs using Median/Mean of the column

In [17]:
# using median
df['Salary'].fillna(df['Salary'].median())

0       97308.0
1       61933.0
2      130590.0
3       90370.0
4      101004.0
         ...   
995    132483.0
996     42392.0
997     96914.0
998     60500.0
999    129949.0
Name: Salary, Length: 1000, dtype: float64

In [18]:
#using mean
df['Salary'].fillna(int(df['Salary'].mean()))

0       97308.0
1       61933.0
2      130590.0
3       90522.0
4      101004.0
         ...   
995    132483.0
996     42392.0
997     96914.0
998     60500.0
999    129949.0
Name: Salary, Length: 1000, dtype: float64

### Using the replace method
The replace method is a more generic form of the fillna method. Here, we specify both the value to be replaced and the replacement value.

In [19]:
# will replace NaN value in Salary with value 0  
df['Salary'].replace(to_replace = np.nan, value = 0)

0       97308.0
1       61933.0
2      130590.0
3           0.0
4      101004.0
         ...   
995    132483.0
996     42392.0
997     96914.0
998     60500.0
999    129949.0
Name: Salary, Length: 1000, dtype: float64

### Using the interpolate method
`interpolate()` function is used to fill NaN values using various interpolation techniques. 

Let us interpolate the missing values using the Linear Interpolation method

In [20]:
df['Salary'].interpolate(method='linear', direction = 'forward')

0       97308.0
1       61933.0
2      130590.0
3      115797.0
4      101004.0
         ...   
995    132483.0
996     42392.0
997     96914.0
998     60500.0
999    129949.0
Name: Salary, Length: 1000, dtype: float64